In [2]:
import pandas as pd
from glob import glob
from simpledbf import Dbf5
import os
import pyproj
import numpy as np

# GlaThiDa templates

In [5]:
templ_t = pd.read_excel('C:\\Users\\jlandman\\Desktop\\GlaThiDa_2-0_DataSubmissionForm_for_pandas.xls', sheetname='T - GLACIER THICKNESS OVERVIEW') 
templ_tt = pd.read_excel('C:\\Users\\jlandman\\Desktop\\GlaThiDa_2-0_DataSubmissionForm_for_pandas.xls', sheetname='TT - GL. THICKNESS DISTRIBUTION')
templ_ttt = pd.read_excel('C:\\Users\\jlandman\\Desktop\\GlaThiDa_2-0_DataSubmissionForm_for_pandas.xls', sheetname='TTT - GL. THICKNESS POINT DATA')

# New GlaThiDa IDs for the treated glaciers

In [84]:
gtd_ids = {'brewster': 2077,
          'north': 2078,
          'south':2079,
          'starbuck_gprt':2080,
          'starbuck_icebr':2081,
          'tasman': 2082,
          'urumqui':2083,
          'west_washamawapta':2084}

# Brewster glacier

In [9]:
nzgd = pyproj.Proj(init='epsg:2193') # NZGD2000_New_Zealand_Transverse_Mercator_2000
latlon = pyproj.Proj(init='epsg:4326')  #WGS84 lat/lon

In [20]:
# constants
brew = {'punit': 'NZ',
'gname': 'BREWSTER',
'src_id':'',
'src_g_id':'',
'survey_date': '19979999',
'dem_date': '19860399',
'gid': gtd_ids['brewster'],
'lat': -44.07,
'lon': 169.43,
'area': 2.62,
'mean_sl': np.nan,
'mean_th': np.nan,
'mean_th_unc': np.nan,
'max_th': np.nan,
'max_th_unc': np.nan,
'surv_meth': 'GPRt',
'surv_meth_det': '',
'no_points': 588,
'no_prfls': 4,
'length_prfls': np.nan,
'interp_meth':'',
'investig':'Bob JACOBEL and Ian OWENS; compiled by Brian ANDERSON',
'spons_ag':'',
'ref':'Willis, I. et al. (2009). Hydrological Processes, 23(3), p.384-396. DOI: 10.1002/hyp.7146',
'dflag':np.nan,
'remarks_t':'Source_ID: Outline and DEM from aerial photography for the NZMS260 mapping series flown in 3/1986. Lat/lon from WGMS FoG2015. Data unpublished, but used in given reference.',
'remarks_ttt':''
       }

In [38]:
brewster_path = 'C:\\Users\\jlandman\\Desktop\\newData\\WG_Farinotti_data_package\\brewster\\brewster_jacobel_owens_bed_xyz.csv'

# read original data
brew_dat = pd.read_csv(brewster_path)

# convert New Real TM to WGS84 lat/lon
xs = brew_dat['nztm_easting (m)'].values
ys = brew_dat[' nztm_northing (m)'].values
x1,y1 = nzgd(xs, ys)
lons, lats = pyproj.transform(nzgd,latlon,xs,ys)

# fill T table
brew_t = templ_t.copy()
brew_t.loc[0,'GlaThiDa_ID'] = brew['gid']
print(brew['gid'])
brew_t.loc[0,'POLITICAL_UNIT'] = brew['punit']
brew_t.loc[0,'GLACIER_NAME'] = brew['gname']
brew_t.loc[0,'SOURCE_ID'] = brew['src_id']
brew_t.loc[0,'GLACIER_ID'] = brew['src_g_id']
brew_t.loc[0,'LAT'] = brew['lat']
brew_t.loc[0,'LON'] = brew['lon']
brew_t.loc[0,'SURVEY_DATE'] = brew['survey_date']
brew_t.loc[0,'DEM_DATE'] = brew['dem_date']
brew_t.loc[0,'AREA'] = brew['area']
brew_t.loc[0,'MEAN_SLOPE'] = brew['mean_sl']
brew_t.loc[0,'MEAN_THICKNESS'] = brew['mean_th']
brew_t.loc[0,'MEAN_THICKNESS_UNCERTAINTY'] = brew['mean_th_unc']
brew_t.loc[0,'MAXIMUM_THICKNESS'] = brew['max_th']
brew_t.loc[0,'MAX_THICKNESS_UNCERTAINTY'] = brew['max_th_unc']
brew_t.loc[0,'SURVEY_METHOD'] = brew['surv_meth']
brew_t.loc[0,'SURVEY_METHOD_DETAILS'] = brew['surv_meth_det']
brew_t.loc[0,'NUMBER_OF_SURVEY_POINTS'] = brew['no_points']
brew_t.loc[0,'NUMBER_OF_SURVEY_PROFILES'] = brew['no_prfls']
brew_t.loc[0,'TOTAL_LENGTH_OF_SURVEY_PROFILES'] = brew['length_prfls']
brew_t.loc[0,'INTERPOLATION_METHOD'] = brew['interp_meth']
brew_t.loc[0,'INVESTIGATOR'] = brew['investig']
brew_t.loc[0,'SPONSORING_AGENCY'] = brew['spons_ag']
brew_t.loc[0,'REFERENCES'] = brew['ref']
brew_t.loc[0,'DATA_FLAG'] = brew['dflag']
brew_t.loc[0,'REMARKS'] = brew['remarks_t']
print(brew_t)

# fill TTT table
brew_ttt = templ_ttt.copy()
brew_ttt['POINT_ID'] = range(1,len(brew_dat)+1)
brew_ttt['POINT_LAT'] = lats
brew_ttt['POINT_LON'] = lons
brew_ttt['ELEVATION'] = (brew_dat[' surface_elevation (m asl)']).map('{:.0f}'.format)
brew_ttt['THICKNESS'] = (brew_dat[' ice_thickness (m)']).map('{:.1f}'.format)
brew_ttt['THICKNESS_UNCERTAINTY'] = np.nan
brew_ttt['GlaThiDa_ID'] = brew['gid']
brew_ttt['POLITICAL_UNIT'] = brew['punit']
brew_ttt['GLACIER_NAME'] = brew['gname']
brew_ttt['SURVEY_DATE'] = brew['survey_date']
brew_ttt['DATA_FLAG'] = brew['dflag']
brew_ttt['REMARKS'] = brew['remarks_ttt'] 

brew_t = brew_t[['GlaThiDa_ID', 'POLITICAL_UNIT', 'GLACIER_NAME', 'SOURCE_ID', 'GLACIER_ID', 'LAT', 'LON', 'SURVEY_DATE', 
                 'DEM_DATE', 'AREA', 'MEAN_SLOPE', 'MEAN_THICKNESS', 'MEAN_THICKNESS_UNCERTAINTY', 'MAXIMUM_THICKNESS', 
                 'MAX_THICKNESS_UNCERTAINTY', 'SURVEY_METHOD', 'SURVEY_METHOD_DETAILS', 'NUMBER_OF_SURVEY_POINTS',
                 'NUMBER_OF_SURVEY_PROFILES', 'TOTAL_LENGTH_OF_SURVEY_PROFILES', 'INTERPOLATION_METHOD', 'INVESTIGATOR', 
                 'SPONSORING_AGENCY', 'REFERENCES', 'DATA_FLAG', 'REMARKS']]
brew_ttt = brew_ttt[['GlaThiDa_ID','POLITICAL_UNIT','GLACIER_NAME','SURVEY_DATE','POINT_ID','POINT_LAT','POINT_LON',
                     'ELEVATION','THICKNESS','THICKNESS_UNCERTAINTY','DATA_FLAG','REMARKS']]

brew_t.to_excel(os.path.join(os.path.dirname(brewster_path),brewster_path.split('.')[0]+'_Johannes_T.xls'), index=False)
brew_ttt.to_excel(os.path.join(os.path.dirname(brewster_path),brewster_path.split('.')[0]+'_Johannes_TTT.xls'), index=False)

KeyError: 'nztm_easting (m)'

# North glacier

In [42]:
utm7n = pyproj.Proj(init='epsg:32607') # UTM 7N (WGS84)
latlon = pyproj.Proj(init='epsg:4326')  #WGS84 lat/lon

In [49]:
# constants
north = {'punit': 'CA',
'gname': 'NORTH',
'src_id':'',
'src_g_id':'',
'survey_date': '20119999',
'dem_date': '20079999',
'gid': gtd_ids['north'],
'lat': 60.911031,
'lon': -139.150803,
'area': 6.9,
'mean_sl': np.nan,
'mean_th': np.nan,
'mean_th_unc': np.nan,
'max_th': np.nan,
'max_th_unc': np.nan,
'surv_meth': 'GPRt/DRIh',
'surv_meth_det': '',
'no_points': 7277,
'no_prfls': np.nan,
'length_prfls': np.nan,
'interp_meth':'',
'investig':'Nat WILSON, Glenn FLOWERS, Laurent MINGO',
'spons_ag':'',
'ref':'Wilson, N. J., Flowers, G. E., & Mingo, L. (2013). Journal of Geophysical Research: Earth Surface, 118(3), pp.1443-1459. DOI:10.1002/jgrf.20096',
'dflag':np.nan,
'remarks_t':'Lat/lon from Google Earth.',
'remarks_ttt':''
       }

thin_value_north = 7

In [50]:
north_path = 'C:\\Users\\jlandman\\Desktop\\newData\\WG_Farinotti_data_package\\north_and_south_glacier\\depth_GL2_080911.xyz'

# read original data
north_dat = pd.read_csv(north_path, header=None, sep='\t') # 0:easting (m), 1:northing (m), 2:depth (m), 3:estimated picking error (m) 

# convert New Real TM to WGS84 lat/lon
xs = north_dat[0].values
ys = north_dat[1].values
x1,y1 = utm7n(xs, ys)
lons, lats = pyproj.transform(utm7n,latlon,xs,ys)

# fill T table
north_t = templ_t.copy()
north_t.loc[0,'GlaThiDa_ID'] = north['gid']
north_t.loc[0,'POLITICAL_UNIT'] = north['punit']
north_t.loc[0,'GLACIER_NAME'] = north['gname']
north_t.loc[0,'SOURCE_ID'] = north['src_id']
north_t.loc[0,'GLACIER_ID'] = north['src_g_id']
north_t.loc[0,'LAT'] = north['lat']
north_t.loc[0,'LON'] = north['lon']
north_t.loc[0,'SURVEY_DATE'] = north['survey_date']
north_t.loc[0,'DEM_DATE'] = north['dem_date']
north_t.loc[0,'AREA'] = north['area']
north_t.loc[0,'MEAN_SLOPE'] = north['mean_sl']
north_t.loc[0,'MEAN_THICKNESS'] = north['mean_th']
north_t.loc[0,'MEAN_THICKNESS_UNCERTAINTY'] = north['mean_th_unc']
north_t.loc[0,'MAXIMUM_THICKNESS'] = north['max_th']
north_t.loc[0,'MAX_THICKNESS_UNCERTAINTY'] = north['max_th_unc']
north_t.loc[0,'SURVEY_METHOD'] = north['surv_meth']
north_t.loc[0,'SURVEY_METHOD_DETAILS'] = north['surv_meth_det']
north_t.loc[0,'NUMBER_OF_SURVEY_POINTS'] = north['no_points']
north_t.loc[0,'NUMBER_OF_SURVEY_PROFILES'] = north['no_prfls']
north_t.loc[0,'TOTAL_LENGTH_OF_SURVEY_PROFILES'] = north['length_prfls']
north_t.loc[0,'INTERPOLATION_METHOD'] = north['interp_meth']
north_t.loc[0,'INVESTIGATOR'] = north['investig']
north_t.loc[0,'SPONSORING_AGENCY'] = north['spons_ag']
north_t.loc[0,'REFERENCES'] = north['ref']
north_t.loc[0,'DATA_FLAG'] = north['dflag']
north_t.loc[0,'REMARKS'] = north['remarks_t']


# fill TTT table and thin data
north_ttt = templ_ttt.copy()
north_ttt['POINT_ID'] = range(1,len(north_dat)+1)
north_ttt['POINT_LAT'] = lats
north_ttt['POINT_LON'] = lons
north_ttt['ELEVATION'] = np.nan
north_ttt['THICKNESS'] = (north_dat[2]).map('{:.1f}'.format)
north_ttt['THICKNESS_UNCERTAINTY'] = np.nan

north_ttt = north_ttt.groupby(north_ttt.index // thin_values_north).mean() # average over every x values as specified above
north_ttt['GlaThiDa_ID'] = north['gid']
north_ttt['POLITICAL_UNIT'] = north['punit']
north_ttt['GLACIER_NAME'] = north['gname']
north_ttt['SURVEY_DATE'] = north['survey_date']
north_ttt['DATA_FLAG'] = north['dflag']
north_ttt['REMARKS'] = north['remarks_ttt'] 

north_t = north_t[['GlaThiDa_ID', 'POLITICAL_UNIT', 'GLACIER_NAME', 'SOURCE_ID', 'GLACIER_ID', 'LAT', 'LON', 'SURVEY_DATE', 
                 'DEM_DATE', 'AREA', 'MEAN_SLOPE', 'MEAN_THICKNESS', 'MEAN_THICKNESS_UNCERTAINTY', 'MAXIMUM_THICKNESS', 
                 'MAX_THICKNESS_UNCERTAINTY', 'SURVEY_METHOD', 'SURVEY_METHOD_DETAILS', 'NUMBER_OF_SURVEY_POINTS',
                 'NUMBER_OF_SURVEY_PROFILES', 'TOTAL_LENGTH_OF_SURVEY_PROFILES', 'INTERPOLATION_METHOD', 'INVESTIGATOR', 
                 'SPONSORING_AGENCY', 'REFERENCES', 'DATA_FLAG', 'REMARKS']]
north_ttt = north_ttt[['GlaThiDa_ID','POLITICAL_UNIT','GLACIER_NAME','SURVEY_DATE','POINT_ID','POINT_LAT','POINT_LON',
                     'ELEVATION','THICKNESS','THICKNESS_UNCERTAINTY','DATA_FLAG','REMARKS']]

north_t.to_excel(os.path.join(os.path.dirname(north_path),north_path.split('.')[0]+'_Johannes_T.xls'), index=False)
north_ttt.to_excel(os.path.join(os.path.dirname(north_path),north_path.split('.')[0]+'_Johannes_TTT.xls'), index=False)

# South glacier

In [ ]:
utm7n = pyproj.Proj(init='epsg:32607') # UTM 7N (WGS84)
latlon = pyproj.Proj(init='epsg:4326')  #WGS84 lat/lon

In [ ]:
# constants
north = {'punit': 'CA',
'gname': 'SOUTH',
'src_id':'',
'src_g_id':'',
'survey_date': '20119999',
'dem_date': '20079999',
'gid': gtd_ids['south'],
'lat': 60.817771,
'lon': -139.124004,
'area': 5.3,
'mean_sl': np.nan,
'mean_th': np.nan,
'mean_th_unc': np.nan,
'max_th': np.nan,
'max_th_unc': np.nan,
'surv_meth': 'GPRt/DRIh',
'surv_meth_det': '',
'no_points': 9618,
'no_prfls': np.nan,
'length_prfls': np.nan,
'interp_meth':'',
'investig':'Nat WILSON, Glenn FLOWERS, Laurent MINGO',
'spons_ag':'',
'ref':'Wilson, N. J., Flowers, G. E., & Mingo, L. (2013). Journal of Geophysical Research: Earth Surface, 118(3), pp.1443-1459. DOI:10.1002/jgrf.20096',
'dflag':np.nan,
'remarks_t':'Lat/lon from Google Earth.',
'remarks_ttt':''
       }

thin_value_south = 10

In [ ]:
south_path = 'C:\\Users\\jlandman\\Desktop\\newData\\WG_Farinotti_data_package\\north_and_south_glacier\\depth_GL1_080911.xyz'

# read original data
south_dat = pd.read_csv(south_path, header=None, sep='\t') # 0:easting (m), 1:northing (m), 2:depth (m), 3:estimated picking error (m) 

# convert New Real TM to WGS84 lat/lon
xs = south_dat[0].values
ys = south_dat[1].values
x1,y1 = utm7n(xs, ys)
lons, lats = pyproj.transform(utm7n,latlon,xs,ys)

# fill T table
south_t = templ_t.copy()
south_t.loc[0,'GlaThiDa_ID'] = south['gid']
south_t.loc[0,'POLITICAL_UNIT'] = south['punit']
south_t.loc[0,'GLACIER_NAME'] = south['gname']
south_t.loc[0,'SOURCE_ID'] = south['src_id']
south_t.loc[0,'GLACIER_ID'] = south['src_g_id']
south_t.loc[0,'LAT'] = south['lat']
south_t.loc[0,'LON'] = south['lon']
south_t.loc[0,'SURVEY_DATE'] = south['survey_date']
south_t.loc[0,'DEM_DATE'] = south['dem_date']
south_t.loc[0,'AREA'] = south['area']
south_t.loc[0,'MEAN_SLOPE'] = south['mean_sl']
south_t.loc[0,'MEAN_THICKNESS'] = south['mean_th']
south_t.loc[0,'MEAN_THICKNESS_UNCERTAINTY'] = south['mean_th_unc']
south_t.loc[0,'MAXIMUM_THICKNESS'] = south['max_th']
south_t.loc[0,'MAX_THICKNESS_UNCERTAINTY'] = south['max_th_unc']
south_t.loc[0,'SURVEY_METHOD'] = south['surv_meth']
south_t.loc[0,'SURVEY_METHOD_DETAILS'] = south['surv_meth_det']
south_t.loc[0,'NUMBER_OF_SURVEY_POINTS'] = south['no_points']
south_t.loc[0,'NUMBER_OF_SURVEY_PROFILES'] = south['no_prfls']
south_t.loc[0,'TOTAL_LENGTH_OF_SURVEY_PROFILES'] = south['length_prfls']
south_t.loc[0,'INTERPOLATION_METHOD'] = south['interp_meth']
south_t.loc[0,'INVESTIGATOR'] = south['investig']
south_t.loc[0,'SPONSORING_AGENCY'] = south['spons_ag']
south_t.loc[0,'REFERENCES'] = south['ref']
south_t.loc[0,'DATA_FLAG'] = south['dflag']
south_t.loc[0,'REMARKS'] = south['remarks_t']


# fill TTT table and thin data
south_ttt = templ_ttt.copy()
south_ttt['POINT_ID'] = range(1,len(south_dat)+1)
south_ttt['POINT_LAT'] = lats
south_ttt['POINT_LON'] = lons
south_ttt['ELEVATION'] = np.nan
south_ttt['THICKNESS'] = (south_dat[2]).map('{:.1f}'.format)
south_ttt['THICKNESS_UNCERTAINTY'] = np.nan

south_ttt = south_ttt.groupby(south_ttt.index // thin_values_south).mean() # average over every x values as specified above
south_ttt['GlaThiDa_ID'] = south['gid']
south_ttt['POLITICAL_UNIT'] = south['punit']
south_ttt['GLACIER_NAME'] = south['gname']
south_ttt['SURVEY_DATE'] = south['survey_date']
south_ttt['DATA_FLAG'] = south['dflag']
south_ttt['REMARKS'] = south['remarks_ttt'] 

south_t = south_t[['GlaThiDa_ID', 'POLITICAL_UNIT', 'GLACIER_NAME', 'SOURCE_ID', 'GLACIER_ID', 'LAT', 'LON', 'SURVEY_DATE', 
                 'DEM_DATE', 'AREA', 'MEAN_SLOPE', 'MEAN_THICKNESS', 'MEAN_THICKNESS_UNCERTAINTY', 'MAXIMUM_THICKNESS', 
                 'MAX_THICKNESS_UNCERTAINTY', 'SURVEY_METHOD', 'SURVEY_METHOD_DETAILS', 'NUMBER_OF_SURVEY_POINTS',
                 'NUMBER_OF_SURVEY_PROFILES', 'TOTAL_LENGTH_OF_SURVEY_PROFILES', 'INTERPOLATION_METHOD', 'INVESTIGATOR', 
                 'SPONSORING_AGENCY', 'REFERENCES', 'DATA_FLAG', 'REMARKS']]
south_ttt = south_ttt[['GlaThiDa_ID','POLITICAL_UNIT','GLACIER_NAME','SURVEY_DATE','POINT_ID','POINT_LAT','POINT_LON',
                     'ELEVATION','THICKNESS','THICKNESS_UNCERTAINTY','DATA_FLAG','REMARKS']]

south_t.to_excel(os.path.join(os.path.dirname(south_path),south_path.split('.')[0]+'_Johannes_T.xls'), index=False)
south_ttt.to_excel(os.path.join(os.path.dirname(south_path),south_path.split('.')[0]+'_Johannes_TTT.xls'), index=False)

# Starbuck glacier GPRt and ICEBRIDGE measurements

# Starbuck GPRt

In [60]:
aqps = pyproj.Proj(init='epsg:3031') # Antarctica polar stereographic
latlon = pyproj.Proj(init='epsg:4326')  #WGS84 lat/lon

In [61]:
# constants
starb = {'punit': 'AQ',
'gname': 'STARBUCK',
'src_id':'',
'src_g_id':'',
'survey_date': '20121299',
'dem_date': '20079999',
'gid': gtd_ids['starbuck_gprt'],
'lat': -65.614439,
'lon': -62.418465,
'area': 258,
'mean_sl': np.nan,
'mean_th': 312,
'mean_th_unc': 30,
'max_th': 1020,
'max_th_unc': np.nan,
'surv_meth': 'GPRt',
'surv_meth_det': 'Deep-Look Radio-Echo Sounder (DELORES), 1-4 MHz range (3MHz central frequ.)',
'no_points': 1315,
'no_prfls': 41,
'length_prfls': 90,
'interp_meth':'',
'investig':'Daniel FARINOTTI and colleagues',
'spons_ag':'',
'ref':'Farinotti, D. et al. (2014). Annals of Glaciology, 55(67), pp. 22-28. DOI:10.3189/2014AoG67A025',
'dflag':np.nan,
'remarks_t':'DEM from Cook et al. (2012) (doi: 10.5194/essd-4-129-2012)',
'remarks_ttt':''
       }


In [69]:
starb_path = 'C:\\Users\\jlandman\\Desktop\\newData\\WG_Farinotti_data_package\\starbuck\\starbuck_RES_data_including_MCoRDS.txt'

# read original data
starb_dat = pd.read_csv(starb_path, comment='#', delim_whitespace=True) 

# select here the GPRt measurements
starb_dat = starb_dat[starb_dat.orgnm != 'icebridge']

# convert New Real TM to WGS84 lat/lon
xs = starb_dat['lon'].values
ys = starb_dat['lat'].values
x1,y1 = aqps(xs, ys)
lons, lats = pyproj.transform(aqps,latlon,xs,ys)

# fill T table
starb_t = templ_t.copy()
starb_t.loc[0,'GlaThiDa_ID'] = starb['gid']
starb_t.loc[0,'POLITICAL_UNIT'] = starb['punit']
starb_t.loc[0,'GLACIER_NAME'] = starb['gname']
starb_t.loc[0,'SOURCE_ID'] = starb['src_id']
starb_t.loc[0,'GLACIER_ID'] = starb['src_g_id']
starb_t.loc[0,'LAT'] = starb['lat']
starb_t.loc[0,'LON'] = starb['lon']
starb_t.loc[0,'SURVEY_DATE'] = starb['survey_date']
starb_t.loc[0,'DEM_DATE'] = starb['dem_date']
starb_t.loc[0,'AREA'] = starb['area']
starb_t.loc[0,'MEAN_SLOPE'] = starb['mean_sl']
starb_t.loc[0,'MEAN_THICKNESS'] = starb['mean_th']
starb_t.loc[0,'MEAN_THICKNESS_UNCERTAINTY'] = starb['mean_th_unc']
starb_t.loc[0,'MAXIMUM_THICKNESS'] = starb['max_th']
starb_t.loc[0,'MAX_THICKNESS_UNCERTAINTY'] = starb['max_th_unc']
starb_t.loc[0,'SURVEY_METHOD'] = starb['surv_meth']
starb_t.loc[0,'SURVEY_METHOD_DETAILS'] = starb['surv_meth_det']
starb_t.loc[0,'NUMBER_OF_SURVEY_POINTS'] = starb['no_points']
starb_t.loc[0,'NUMBER_OF_SURVEY_PROFILES'] = starb['no_prfls']
starb_t.loc[0,'TOTAL_LENGTH_OF_SURVEY_PROFILES'] = starb['length_prfls']
starb_t.loc[0,'INTERPOLATION_METHOD'] = starb['interp_meth']
starb_t.loc[0,'INVESTIGATOR'] = starb['investig']
starb_t.loc[0,'SPONSORING_AGENCY'] = starb['spons_ag']
starb_t.loc[0,'REFERENCES'] = starb['ref']
starb_t.loc[0,'DATA_FLAG'] = starb['dflag']
starb_t.loc[0,'REMARKS'] = starb['remarks_t']


# fill TTT table
starb_ttt = templ_ttt.copy()
starb_ttt['POINT_ID'] = range(1,len(starb_dat)+1)
starb_ttt['POINT_LAT'] = lats
starb_ttt['POINT_LON'] = lons
starb_ttt['ELEVATION'] = (starb_dat['z_surf']).map('{:.0f}'.format)
starb_ttt['THICKNESS'] = (starb_dat['ice_th']).map('{:.1f}'.format)
starb_ttt['THICKNESS_UNCERTAINTY'] = np.nan
starb_ttt['GlaThiDa_ID'] = starb['gid']
starb_ttt['POLITICAL_UNIT'] = starb['punit']
starb_ttt['GLACIER_NAME'] = starb['gname']
starb_ttt['SURVEY_DATE'] = starb['survey_date']
starb_ttt['DATA_FLAG'] = starb['dflag']
starb_ttt['REMARKS'] = starb['remarks_ttt'] 

starb_t = starb_t[['GlaThiDa_ID', 'POLITICAL_UNIT', 'GLACIER_NAME', 'SOURCE_ID', 'GLACIER_ID', 'LAT', 'LON', 'SURVEY_DATE', 
                 'DEM_DATE', 'AREA', 'MEAN_SLOPE', 'MEAN_THICKNESS', 'MEAN_THICKNESS_UNCERTAINTY', 'MAXIMUM_THICKNESS', 
                 'MAX_THICKNESS_UNCERTAINTY', 'SURVEY_METHOD', 'SURVEY_METHOD_DETAILS', 'NUMBER_OF_SURVEY_POINTS',
                 'NUMBER_OF_SURVEY_PROFILES', 'TOTAL_LENGTH_OF_SURVEY_PROFILES', 'INTERPOLATION_METHOD', 'INVESTIGATOR', 
                 'SPONSORING_AGENCY', 'REFERENCES', 'DATA_FLAG', 'REMARKS']]
starb_ttt = starb_ttt[['GlaThiDa_ID','POLITICAL_UNIT','GLACIER_NAME','SURVEY_DATE','POINT_ID','POINT_LAT','POINT_LON',
                     'ELEVATION','THICKNESS','THICKNESS_UNCERTAINTY','DATA_FLAG','REMARKS']]

starb_t.to_excel(os.path.join(os.path.dirname(starb_path),starb_path.split('.')[0]+'_GPRt_Johannes_T.xls'), index=False)
starb_ttt.to_excel(os.path.join(os.path.dirname(starb_path),starb_path.split('.')[0]+'_GPRt_Johannes_TTT.xls'), index=False)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\jlandman\\Desktop\\newData\\WG_Farinotti_data_package\\starbuck\\starbuck_RES_data_including_MCoRDS_GPRt_Johannes_TTT.xls'

# Starbuck glacier ICEBRIDGE

In [77]:
# constants
starb_i = {'punit': 'AQ',
'gname': 'STARBUCK',
'src_id':'',
'src_g_id':'',
'survey_date': '20111014',
'dem_date': '20079999',
'gid': gtd_ids['starbuck_icebr'],
'lat': -65.614439,
'lon': -62.418465,
'area': 258,
'mean_sl': np.nan,
'mean_th': np.nan,
'mean_th_unc': np.nan,
'max_th': np.nan,
'max_th_unc': np.nan,
'surv_meth': 'OTH',
'surv_meth_det': 'Deep-Look Radio-Echo Sounder (DELORES), 1-4 MHz range (3MHz central frequ.)',
'no_points': 103,
'no_prfls': np.nan,
'length_prfls': np.nan,
'interp_meth':'',
'investig':'',
'spons_ag':'',
'ref':'Farinotti, D. et al. (2014). Annals of Glaciology, 55(67), pp. 22-28. DOI:10.3189/2014AoG67A025',
'dflag':np.nan,
'remarks_t':'Survey Method: NASA IceBridge project (MCoRDS sensor). Data used as base for cross-validation in given reference. DEM from Cook et al. (2012) (doi: 10.5194/essd-4-129-2012)',
'remarks_ttt':''
       }

In [79]:
starb_path = 'C:\\Users\\jlandman\\Desktop\\newData\\WG_Farinotti_data_package\\starbuck\\starbuck_RES_data_including_MCoRDS.txt'

# read original data
starb_i_dat = pd.read_csv(starb_path, comment='#', delim_whitespace=True) 

# select here the GPRt measurements
starb_i_dat = starb_i_dat[starb_i_dat.orgnm == 'icebridge']

# convert New Real TM to WGS84 lat/lon
xs = starb_i_dat['lon'].values
ys = starb_i_dat['lat'].values
x1,y1 = aqps(xs, ys)
lons, lats = pyproj.transform(aqps,latlon,xs,ys)

# fill T table
starb_i_t = templ_t.copy()
starb_i_t.loc[0,'GlaThiDa_ID'] = starb_i['gid']
starb_i_t.loc[0,'POLITICAL_UNIT'] = starb_i['punit']
starb_i_t.loc[0,'GLACIER_NAME'] = starb_i['gname']
starb_i_t.loc[0,'SOURCE_ID'] = starb_i['src_id']
starb_i_t.loc[0,'GLACIER_ID'] = starb_i['src_g_id']
starb_i_t.loc[0,'LAT'] = starb_i['lat']
starb_i_t.loc[0,'LON'] = starb_i['lon']
starb_i_t.loc[0,'SURVEY_DATE'] = starb_i['survey_date']
starb_i_t.loc[0,'DEM_DATE'] = starb_i['dem_date']
starb_i_t.loc[0,'AREA'] = starb_i['area']
starb_i_t.loc[0,'MEAN_SLOPE'] = starb_i['mean_sl']
starb_i_t.loc[0,'MEAN_THICKNESS'] = starb_i['mean_th']
starb_i_t.loc[0,'MEAN_THICKNESS_UNCERTAINTY'] = starb_i['mean_th_unc']
starb_i_t.loc[0,'MAXIMUM_THICKNESS'] = starb_i['max_th']
starb_i_t.loc[0,'MAX_THICKNESS_UNCERTAINTY'] = starb_i['max_th_unc']
starb_i_t.loc[0,'SURVEY_METHOD'] = starb_i['surv_meth']
starb_i_t.loc[0,'SURVEY_METHOD_DETAILS'] = starb_i['surv_meth_det']
starb_i_t.loc[0,'NUMBER_OF_SURVEY_POINTS'] = starb_i['no_points']
starb_i_t.loc[0,'NUMBER_OF_SURVEY_PROFILES'] = starb_i['no_prfls']
starb_i_t.loc[0,'TOTAL_LENGTH_OF_SURVEY_PROFILES'] = starb_i['length_prfls']
starb_i_t.loc[0,'INTERPOLATION_METHOD'] = starb_i['interp_meth']
starb_i_t.loc[0,'INVESTIGATOR'] = starb_i['investig']
starb_i_t.loc[0,'SPONSORING_AGENCY'] = starb_i['spons_ag']
starb_i_t.loc[0,'REFERENCES'] = starb_i['ref']
starb_i_t.loc[0,'DATA_FLAG'] = starb_i['dflag']
starb_i_t.loc[0,'REMARKS'] = starb_i['remarks_t']


# fill TTT table
starb_i_ttt = templ_ttt.copy()
starb_i_ttt['POINT_ID'] = range(1,len(starb_i_dat)+1)
starb_i_ttt['POINT_LAT'] = lats
starb_i_ttt['POINT_LON'] = lons
starb_i_ttt['ELEVATION'] = (starb_i_dat['z_surf']).map('{:.0f}'.format)
starb_i_ttt['THICKNESS'] = (starb_i_dat['ice_th']).map('{:.1f}'.format)
starb_i_ttt['THICKNESS_UNCERTAINTY'] = np.nan
starb_i_ttt['GlaThiDa_ID'] = starb_i['gid']
starb_i_ttt['POLITICAL_UNIT'] = starb_i['punit']
starb_i_ttt['GLACIER_NAME'] = starb_i['gname']
starb_i_ttt['SURVEY_DATE'] = starb_i['survey_date']
starb_i_ttt['DATA_FLAG'] = starb_i['dflag']
starb_i_ttt['REMARKS'] = starb_i['remarks_ttt'] 

starb_i_t = starb_i_t[['GlaThiDa_ID', 'POLITICAL_UNIT', 'GLACIER_NAME', 'SOURCE_ID', 'GLACIER_ID', 'LAT', 'LON', 'SURVEY_DATE', 
                 'DEM_DATE', 'AREA', 'MEAN_SLOPE', 'MEAN_THICKNESS', 'MEAN_THICKNESS_UNCERTAINTY', 'MAXIMUM_THICKNESS', 
                 'MAX_THICKNESS_UNCERTAINTY', 'SURVEY_METHOD', 'SURVEY_METHOD_DETAILS', 'NUMBER_OF_SURVEY_POINTS',
                 'NUMBER_OF_SURVEY_PROFILES', 'TOTAL_LENGTH_OF_SURVEY_PROFILES', 'INTERPOLATION_METHOD', 'INVESTIGATOR', 
                 'SPONSORING_AGENCY', 'REFERENCES', 'DATA_FLAG', 'REMARKS']]
starb_i_ttt = starb_i_ttt[['GlaThiDa_ID','POLITICAL_UNIT','GLACIER_NAME','SURVEY_DATE','POINT_ID','POINT_LAT','POINT_LON',
                     'ELEVATION','THICKNESS','THICKNESS_UNCERTAINTY','DATA_FLAG','REMARKS']]

starb_i_t.to_excel(os.path.join(os.path.dirname(starb_path),starb_path.split('.')[0]+'_IceBridge_Johannes_T.xls'), index=False)
starb_i_ttt.to_excel(os.path.join(os.path.dirname(starb_path),starb_path.split('.')[0]+'_IceBridge_Johannes_TTT.xls'), index=False)

# Tasman Glacier

In [85]:
nzgd = pyproj.Proj(init='epsg:2193') # NZGD2000_New_Zealand_Transverse_Mercator_2000
latlon = pyproj.Proj(init='epsg:4326')  #WGS84 lat/lon

In [86]:
# constants
tasman = {'punit': 'NZ',
'gname': 'TASMAN',
'src_id':'',
'src_g_id':'',
'survey_date': '',
'dem_date': '19869999',
'gid': gtd_ids['tasman'],
'lat':  -43.52,
'lon': 170.32,
'area': 100.3,
'mean_sl': np.nan,
'mean_th': np.nan,
'mean_th_unc': np.nan,
'max_th': np.nan,
'max_th_unc': np.nan,
'surv_meth': 'SEI',
'surv_meth_det': 'Two generalised cross profiles of ice thickness are inferred from reflection seismics.  x, y and ice thickness values are taken at points spaced along the published profiles (Figure 18, p23). A map of interpolated ice thickness (Figure 19, p26), but it is only based on measurements at the two widely-spaced profiles',
'no_points': np.nan,
'no_prfls': 2,
'length_prfls': 3,
'interp_meth':'',
'investig':'',
'spons_ag':'',
'ref':'Anderton, P. W. (1975). Hydrological Research Annual Report 33. Ministry of Works and Development.',
'dflag':np.nan,
'remarks_t':'No mean/max thickness given as there are only highly varying estimates for two cross-sections of the glacier in the reference.',
'remarks_ttt':''
       }

In [87]:
tasman_path = 'C:\\Users\\jlandman\\Desktop\\newData\\WG_Farinotti_data_package\\tasman\\tasman_anderton_bed_xyz.csv'

# read original data
tasman_dat = pd.read_csv(tasman_path)

# convert New Real TM to WGS84 lat/lon
xs = tasman_dat['nztm_easting (m)'].values
ys = tasman_dat[' nztm_northing (m)'].values
x1,y1 = nzgd(xs, ys)
lons, lats = pyproj.transform(nzgd,latlon,xs,ys)

# fill T table
tasman_t = templ_t.copy()
tasman_t.loc[0,'GlaThiDa_ID'] = tasman['gid']
tasman_t.loc[0,'POLITICAL_UNIT'] = tasman['punit']
tasman_t.loc[0,'GLACIER_NAME'] = tasman['gname']
tasman_t.loc[0,'SOURCE_ID'] = tasman['src_id']
tasman_t.loc[0,'GLACIER_ID'] = tasman['src_g_id']
tasman_t.loc[0,'LAT'] = tasman['lat']
tasman_t.loc[0,'LON'] = tasman['lon']
tasman_t.loc[0,'SURVEY_DATE'] = tasman['survey_date']
tasman_t.loc[0,'DEM_DATE'] = tasman['dem_date']
tasman_t.loc[0,'AREA'] = tasman['area']
tasman_t.loc[0,'MEAN_SLOPE'] = tasman['mean_sl']
tasman_t.loc[0,'MEAN_THICKNESS'] = tasman['mean_th']
tasman_t.loc[0,'MEAN_THICKNESS_UNCERTAINTY'] = tasman['mean_th_unc']
tasman_t.loc[0,'MAXIMUM_THICKNESS'] = tasman['max_th']
tasman_t.loc[0,'MAX_THICKNESS_UNCERTAINTY'] = tasman['max_th_unc']
tasman_t.loc[0,'SURVEY_METHOD'] = tasman['surv_meth']
tasman_t.loc[0,'SURVEY_METHOD_DETAILS'] = tasman['surv_meth_det']
tasman_t.loc[0,'NUMBER_OF_SURVEY_POINTS'] = tasman['no_points']
tasman_t.loc[0,'NUMBER_OF_SURVEY_PROFILES'] = tasman['no_prfls']
tasman_t.loc[0,'TOTAL_LENGTH_OF_SURVEY_PROFILES'] = tasman['length_prfls']
tasman_t.loc[0,'INTERPOLATION_METHOD'] = tasman['interp_meth']
tasman_t.loc[0,'INVESTIGATOR'] = tasman['investig']
tasman_t.loc[0,'SPONSORING_AGENCY'] = tasman['spons_ag']
tasman_t.loc[0,'REFERENCES'] = tasman['ref']
tasman_t.loc[0,'DATA_FLAG'] = tasman['dflag']
tasman_t.loc[0,'REMARKS'] = tasman['remarks_t']

# fill TTT table
tasman_ttt = templ_ttt.copy()
tasman_ttt['POINT_ID'] = range(1,len(tasman_dat)+1)
tasman_ttt['POINT_LAT'] = lats
tasman_ttt['POINT_LON'] = lons
tasman_ttt['ELEVATION'] = (tasman_dat[' surface_elevation (m asl)']).map('{:.0f}'.format)
tasman_ttt['THICKNESS'] = (tasman_dat[' ice_thickness (m)']).map('{:.1f}'.format)
tasman_ttt['THICKNESS_UNCERTAINTY'] = np.nan
tasman_ttt['GlaThiDa_ID'] = tasman['gid']
tasman_ttt['POLITICAL_UNIT'] = tasman['punit']
tasman_ttt['GLACIER_NAME'] = tasman['gname']
tasman_ttt['SURVEY_DATE'] = tasman['survey_date']
tasman_ttt['DATA_FLAG'] = tasman['dflag']
tasman_ttt['REMARKS'] = tasman['remarks_ttt'] 

tasman_t = tasman_t[['GlaThiDa_ID', 'POLITICAL_UNIT', 'GLACIER_NAME', 'SOURCE_ID', 'GLACIER_ID', 'LAT', 'LON', 'SURVEY_DATE', 
                 'DEM_DATE', 'AREA', 'MEAN_SLOPE', 'MEAN_THICKNESS', 'MEAN_THICKNESS_UNCERTAINTY', 'MAXIMUM_THICKNESS', 
                 'MAX_THICKNESS_UNCERTAINTY', 'SURVEY_METHOD', 'SURVEY_METHOD_DETAILS', 'NUMBER_OF_SURVEY_POINTS',
                 'NUMBER_OF_SURVEY_PROFILES', 'TOTAL_LENGTH_OF_SURVEY_PROFILES', 'INTERPOLATION_METHOD', 'INVESTIGATOR', 
                 'SPONSORING_AGENCY', 'REFERENCES', 'DATA_FLAG', 'REMARKS']]
tasman_ttt = tasman_ttt[['GlaThiDa_ID','POLITICAL_UNIT','GLACIER_NAME','SURVEY_DATE','POINT_ID','POINT_LAT','POINT_LON',
                     'ELEVATION','THICKNESS','THICKNESS_UNCERTAINTY','DATA_FLAG','REMARKS']]

tasman_t.to_excel(os.path.join(os.path.dirname(tasman_path),tasman_path.split('.')[0]+'_Johannes_T.xls'), index=False)
tasman_ttt.to_excel(os.path.join(os.path.dirname(tasman_path),tasman_path.split('.')[0]+'_Johannes_TTT.xls'), index=False)